# <span style="color: purple;"> <b> PetLove&Co </b> </span>

## **Análise de Churn**

#### O problema

A equipe de assinaturas tem como objetivo reduzir a perda de assinantes. O conceito de “Churn” refere-se a perda de qualquer usuário que assinou o serviço de assinatura da Petlove e o cancelou em algum momento após a contratação.
Ao analisar os dados dos últimos meses, apesar de todas as melhorias de usabilidade da plataforma, o churn vem aumentando.

Este trabalho visa apresentar um resultado final com DOIS pontos principais: os aprendizados obtidos apartir da análise dos dados e uma sugestão como próximos passos para a redução do churn.

## **Configurando ambiente**

In [ ]:
# Instalando pacotes necessários
!pip3 install pandas
!pip3 install matplotlib
!pip3 install plotly

In [97]:
# Importando pacotes necessários
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio

## **Obtendo dados**
Dados obtidos apartir da equipe de engenharia de dados da PetLove

In [129]:
# Carregando o conjunto de dados
df = pd.read_csv("data-test-analytics.csv") # Path do arquivo

## **Visão Geral**
Nesta parte será possível obter insights para descobrir a causa do problema e a resolução do mesmo.

Para tanto, devemos considerar a definição e os tipos das colunas do conjunto de dados:

Nome das colunas | Descrição das colunas
:--------------: | :-------------------:
id | Identificação do cliente
created_at | Data de criação da assinatura
updated_at | Data da última modificação da assinatura
deleted_at | Data de cancelamento da assinatura
name_hash | Nome do usuário (criptografado)
email_hash | Email (criptografado)
address_hash | Endereço (criptografado)
birth_date | Data de aniversário do cliente
status | Status da assinatura
version | Versão da assinatura
city | Cidade do cliente
state | Estado do cliente
neighborhood | Bairro do cliente
last_date_purchase | Data do último pedido que ocorreu pela assinatura
average_ticket | Média de gasto por pedido
items_quantity | Média de itens na assinatura
all_revenue | Total de receita realizado pelo cliente
all_orders | Total de pedidos realizado pelo cliente
recency | Tempo desde a última compra do cliente
marketing_source | Canal de marketing que converteu a assinatura

In [128]:
# Exibindo as primeiras 5 linhas do DataFrame
df.head()

,created_at,updated_at,deleted_at,birth_date,status,version,city,state,last_date_purchase,average_ticket,items_quantity,all_revenue,all_orders,recency,marketing_source
0,2017-08-15 07:05:00,2021-01-14 11:23:00,NaT,1974-07-10,active,2.31.7,Peixoto da Praia,AM,2021-01-14 11:23:00,151.142942,10,906.857651,6,35,crm
1,2019-12-31 21:53:00,2021-01-08 11:23:00,NaT,2040-07-06,paused,3.30.12,Fernandes,RR,2021-01-08 11:23:00,236.991790,4,236.991790,1,41,organic_search
2,2019-03-07 23:46:00,2021-01-07 11:23:00,NaT,2063-03-18,active,3.28.9,Lopes,RR,2021-01-07 11:23:00,211.955597,13,2331.511572,11,42,organic_search
3,2018-07-21 10:17:00,2021-01-10 11:23:00,NaT,1980-11-21,active,3.34.3,Campos do Campo,PE,2021-01-10 11:23:00,204.113227,8,1224.679359,6,39,organic_search
4,2018-06-08 12:09:00,2021-01-18 11:23:00,NaT,2059-07-07,active,3.19.8,das Neves,RJ,2021-01-18 11:23:00,252.940997,9,2023.527980,8,31,crm


In [12]:
# Visuzlizando informações do DataFrame e suas colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   created_at          10000 non-null  object 
 1   updated_at          10000 non-null  object 
 2   deleted_at          505 non-null    object 
 3   birth_date          10000 non-null  object 
 4   status              10000 non-null  object 
 5   version             10000 non-null  object 
 6   city                10000 non-null  object 
 7   state               10000 non-null  object 
 8   last_date_purchase  10000 non-null  object 
 9   average_ticket      10000 non-null  float64
 10  items_quantity      10000 non-null  int64  
 11  all_revenue         10000 non-null  float64
 12  all_orders          10000 non-null  int64  
 13  recency             10000 non-null  int64  
 14  marketing_source    10000 non-null  object 
dtypes: float64(2), int64(3), object(10)
memory usage: 1.1+

In [5]:
# Verificando o número de resgistros duplicados
df.duplicated().sum()

0

Não há registros duplicados

In [6]:
# Verificando valores nulos
df.isnull().sum()

id                       0
created_at               0
updated_at               0
deleted_at            9495
name_hash                0
email_hash               0
address_hash             0
birth_date               0
status                   0
version                  0
city                     0
state                    0
neighborhood             0
last_date_purchase       0
average_ticket           0
items_quantity           0
all_revenue              0
all_orders               0
recency                  0
marketing_source         0
dtype: int64

Valores nulos em "deleted_at" representam clientes que ainda tem sua assinatura ativa

## **Formatando dados**

In [ ]:
# Retiramos as colunas com os dados individuais para melhor visualização
df = df.drop(columns=["id", "name_hash", "email_hash", "address_hash", "neighborhood"], axis=1)
df.head()

In [50]:
# Formatando colunas de datas
df['created_at'] = df['created_at'].astype('datetime64[ns]')
df['updated_at'] = df['updated_at'].astype('datetime64[ns]')
df['deleted_at'] = df['deleted_at'].astype('datetime64[ns]')
df['birth_date'] = df['birth_date'].astype('datetime64[ns]')
df['last_date_purchase'] = df['last_date_purchase'].astype('datetime64[ns]')

## **Análise de dados**
### **Qual a relação geral de clientes com cadastro ativo, pausado e cancelado?**

Visualizando os números com o auxílio do plotly.express:

In [102]:
# Criando gráfico
grafico = px.histogram(df, x="status", color="status", color_discrete_sequence=["purple", "gray", "yellow"], width=400, height=300, 
                        title="Número de clientes em cada status", labels=["Status", "N° de clientes"])
grafico.show()

In [43]:
# Número e porcentagem de clientes por status
display(df["status"].value_counts())

active      8524
paused       971
canceled     505
Name: status, dtype: int64

In [51]:
round((df.groupby(["status"]).size()["canceled"] / df.groupby(["status"]).size()["active"])*100, 2)

5.92

O percentual total de assinaturas canceladas em relação ao número de clientes ativos em todo o período, o "churn", é de 5.92%.

### **Mas como o número de cancelamentos vem crescendo com o passar dos meses?**

Aqui obtemos uma visão mais realista desse número:

In [98]:
# Meses com mais cancelamento
cancelamento_por_data = df.groupby([pd.Grouper(key="deleted_at", freq="M")]).size() # Agrupando por mês

grafico = px.histogram(cancelamento_por_data, x=cancelamento_por_data.index, y=cancelamento_por_data, 
        width=800, height=340, nbins=120, color_discrete_sequence=["purple"], title="Cancelamentos por mês", 
        labels={"deleted_at": "Data do cancelamento", "y": "N° de cancelamentos"}).update_xaxes(categoryorder="total ascending")
grafico.show()

É notável que o número de cancelamentos subiu com a chegada da pandemia do Covid-19, porém existe um grande salto no início de 2020.

In [176]:
# Calculando crescimento do número de cancelamento
round(cancelamento_por_data[-24:].mean()/cancelamento_por_data[0:-24].mean()*100, 2)-100

380.43

Salto esse que significa aproximadamente 380 vezes em relação a média anterior.

### **É possível estabelecer uma relação entre o cancelamento de assinatura e a versão que foi contratada?**

In [99]:
# Versões de assinatura com mais cancelamentos
cancelamento_por_versao = df[["version", "status"]].groupby(["status", "version"]).size()["canceled"].sort_values(ascending=False)

px.histogram(cancelamento_por_versao, x=cancelamento_por_versao.head(10).index, y=cancelamento_por_versao.head(10), 
        width=800, height=340, title="10 versões da assinatura com maior n° de cancelamentos", color_discrete_sequence=["purple"],
        labels={"x":"Versões", "y":"N° de cancelamentos"}).show()

### **Em quais regiões do país temos um maior percentual de assinaturas canceladas?**

Existem estados com número percentual significativamente maior de cancelamentos de assinaturas?

In [163]:
# Estados com mais cancelamentos de planos
cancelamento_por_estado = pd.DataFrame()
cancelamento_por_estado["num cancelados"] = df[["state", "status"]].groupby(["status", "state"]).size()["canceled"] # N° de cancelados
cancelamento_por_estado["num ativos"] = df[["state", "status"]].groupby(["status", "state"]).size()["active"] # N° de ativos
cancelamento_por_estado["porcentagem"] = (cancelamento_por_estado["num cancelados"] / cancelamento_por_estado["num ativos"]) * 100 # Churn

# Criando gráfico
px.histogram(cancelamento_por_estado.sort_values(by="porcentagem",ascending=True), x=cancelamento_por_estado.index, y="porcentagem",
                color_discrete_sequence=["purple"], title="Percentual de cancelamentos por estado",
                labels={"x": "Estados", "porcentagem":"% Percentual de Cancelamentos"}).show()

# Mostrando tabela
display(cancelamento_por_estado.sort_values(by="porcentagem",ascending=False))

,num cancelados,num ativos,porcentagem
state,,,
RS,25,317,7.886435
SE,24,305,7.868852
MT,24,308,7.792208
PA,22,284,7.746479
MA,23,303,7.590759
AL,22,301,7.308970
AM,23,326,7.055215
RR,23,327,7.033639
TO,24,356,6.741573


É possível observar uma discrepância de churn entre diferentes estados

### **Como a falta de recorrência de compras pode levar ao churn?**

In [178]:
# Tempo desde a última compra por n° de cancelamentos
px.histogram(df, x=df["recency"], color="status",width=800, height=340, color_discrete_sequence=["purple", "gray", "yellow"],
                title="Tempo desde a última compra por n° de cancelamentos", 
                labels={"recency":"Tempo desde a última compra", "y":"N° de cancelamentos"}).show()

É possível observar que clientes que não compraram há mais de 60 dias não possuem mais sua assinatura ativada.

### **Existe um meio de conversão com mais porcentagem de churn associado?**

Vamos verificar:

In [197]:
mkt = df.groupby(["status","marketing_source"]).size()["canceled"] / df.groupby(["status","marketing_source"]).size()["active"] *100

px.histogram(mkt, x=mkt.index, y=mkt, width=800, height=340, title="% de cancelamentos por canal de marketing", color_discrete_sequence=["purple"],
        labels={"marketing_source":"Canal de marketing", "y":"% de cancelamentos"}).show()


Maiores taxas de churn relacionadas à conversões sem necessidade de um canal de marketing (none), seguido por redes como Telegram e Whatsapp.

## <span style="color: purple;"> <b> Conclusões </b> </span>

+ Apesar de o número de assinaturas canceladas estar em uma média de aproximadamente 6% em relação ao total do número de clientes com assinatura ativas, os últimos meses registram um aumento significativo na média churn mensal.
+ Existem algumas versões de assinaturas que possuem um número de assinaturas canceladas infimamente maior que as demais.
+ Foi possível identificar que alguns estados brasileiros possuem uma porcentagem maior, uma discrepância, de assinaturas canceladas em relação aos assinantes ativos.
+ Além disso, ao analisar o tempo desde a última compra de cada cliente, foi possível observar que clientes que não compraram há mais de 60 dias não possuem mais sua assinatura ativada.
+ Também é possível identificar taxas de churn mais elevadas relacionadas à conversões sem necessidade de um canal de marketing (none), seguido por redes como Telegram e Whatsapp.

## <span style="color: purple;"> <b> Quais são os próximos passos? </b> </span>

+ 